In [1]:
from sqlalchemy import create_engine


engine = create_engine("sqlite:///example.db", echo=False)

In [2]:
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped, mapped_column


class Base(DeclarativeBase):
    pass


class ToDo(Base):
    __tablename__ = "todos"

    id: Mapped[int] = mapped_column(primary_key=True)
    title: Mapped[str] = mapped_column(nullable=True)
    description: Mapped[str] = mapped_column(nullable=True)

In [3]:
# Create the table in the database
Base.metadata.create_all(engine)

In [4]:
from sqlalchemy.orm import Session


with Session(engine) as session:
    todo = ToDo(title="Buy groceries", description="Buy groceries")
    session.add(todo)
    session.commit()

In [5]:
from sqlalchemy import select


with Session(engine) as session:
    statement = select(ToDo)
    result = session.execute(statement=statement)
    todos = result.scalars().all()

for todo in todos:
    for key, value in todo.__dict__.items():
        print(key, value)


_sa_instance_state <sqlalchemy.orm.state.InstanceState object at 0x000001C3E9948A00>
title Buy groceries
description Buy groceries
id 1
_sa_instance_state <sqlalchemy.orm.state.InstanceState object at 0x000001C3EB5454E0>
title Buy groceries
description Buy groceries
id 2
_sa_instance_state <sqlalchemy.orm.state.InstanceState object at 0x000001C3EB545540>
title Get a haircut
description Get a haircut
id 3
_sa_instance_state <sqlalchemy.orm.state.InstanceState object at 0x000001C3EB5455A0>
title Buy a new laptop
description Buy a new laptop
id 5
_sa_instance_state <sqlalchemy.orm.state.InstanceState object at 0x000001C3EB545600>
title Buy a new car
description Buy a new car
id 6
_sa_instance_state <sqlalchemy.orm.state.InstanceState object at 0x000001C3EB545660>
title Buy a new house
description Buy a new house
id 7
_sa_instance_state <sqlalchemy.orm.state.InstanceState object at 0x000001C3EB5456C0>
title Buy a new boat
description Buy a new boat
id 8
_sa_instance_state <sqlalchemy.orm.s

In [6]:
todos = [
    ToDo(title="Buy groceries", description="Buy groceries"),
    ToDo(title="Get a haircut", description="Get a haircut"),
    ToDo(title="Buy a new phone", description="Buy a new phone"),
    ToDo(title="Buy a new laptop", description="Buy a new laptop"),
    ToDo(title="Buy a new car", description="Buy a new car"),
    ToDo(title="Buy a new house", description="Buy a new house"),
    ToDo(title="Buy a new boat", description="Buy a new boat"),
    ToDo(title="Buy a new plane", description="Buy a new plane"),
    ToDo(title="Buy a new island", description="Buy a new island"),
    ToDo(title="Buy a new moon", description="Buy a new moon"),
    ToDo(title="Buy a new sun", description="Buy a new sun"),
]

with Session(engine) as session:
    session.add_all(todos)
    session.commit()

In [7]:
with Session(engine) as session:
    statement = select(ToDo).filter_by(title="Buy a new phone")
    todo = session.execute(statement=statement).scalar_one()
    todo.title = "Buy a new charger"
    todo.description = "Buy a new charger"
    session.commit()

In [8]:
with Session(engine) as session:
    statement = select(ToDo).filter_by(title="Buy a new charger")
    todo = session.execute(statement=statement).scalar_one()
    session.delete(todo)
    session.commit()

In [9]:
# Core foundations of SQLAlchemy ORM
from sqlalchemy import create_engine


engine = create_engine("sqlite:///example.db", echo=False)

from sqlalchemy import Table, Column, Integer, String, MetaData

metadata = MetaData()

todos_table = Table(
    "todos", 
    metadata, 
    Column("id", Integer, primary_key=True),
    Column("title", String, nullable=True),
    Column("description", String, nullable=True)
)

metadata.create_all(engine)

In [10]:
# Inserting Data
from sqlalchemy import insert

statement = insert(todos_table).values(title="Walk the dog", description="Walk the dog in the park")

with engine.connect() as connection:
    connection.execute(statement)
    connection.commit()


# Selecting Data
from sqlalchemy import select


statement = select(todos_table)

with engine.connect() as connection:
    result = connection.execute(statement)
    todos = result.fetchall()
    for todo in todos:
        print(todo)

(1, 'Buy groceries', 'Buy groceries')
(2, 'Buy groceries', 'Buy groceries')
(3, 'Get a haircut', 'Get a haircut')
(5, 'Buy a new laptop', 'Buy a new laptop')
(6, 'Buy a new car', 'Buy a new car')
(7, 'Buy a new house', 'Buy a new house')
(8, 'Buy a new boat', 'Buy a new boat')
(9, 'Buy a new plane', 'Buy a new plane')
(10, 'Buy a new island', 'Buy a new island')
(11, 'Buy a new moon', 'Buy a new moon')
(12, 'Buy a new sun', 'Buy a new sun')
(13, 'Buy groceries', 'Buy groceries')
(14, 'Buy groceries', 'Buy groceries')
(15, 'Get a haircut', 'Get a haircut')
(17, 'Buy a new laptop', 'Buy a new laptop')
(18, 'Buy a new car', 'Buy a new car')
(19, 'Buy a new house', 'Buy a new house')
(20, 'Buy a new boat', 'Buy a new boat')
(21, 'Buy a new plane', 'Buy a new plane')
(22, 'Buy a new island', 'Buy a new island')
(23, 'Buy a new moon', 'Buy a new moon')
(24, 'Buy a new sun', 'Buy a new sun')
(25, 'Walk the dog', 'Walk the dog in the park')
(26, 'Buy groceries', 'Buy groceries')
(27, 'Buy groc

ORM Basics

In [11]:
import datetime
from sqlalchemy.orm import declarative_base

Base = declarative_base()


from sqlalchemy import Column, Integer, String, DateTime, Boolean

class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=True)
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.datetime.now)


Base.metadata.create_all(engine)

In [12]:
# Sessions (VERY IMPORTANT)
from sqlalchemy.orm import sessionmaker

SessionLocal = sessionmaker(bind=engine)
session = SessionLocal()

In [13]:
users = [
    User(name="Alice"),
    User(name="Bob"),
    User(name="Charlie"),
    User(name="Dave"),
    User(name="Eve"),
]

for user in users:
    session.add(user)
    session.commit()

session.add_all(users)
session.commit()

users = session.query(User).all()

for user in users:
    print(user.id, user.name, user.is_active, user.created_at)

1 Alice True 2026-01-14 22:19:29.236599
2 Bob True 2026-01-14 22:19:29.236599
3 Charlie True 2026-01-14 22:19:29.236599
4 Dave True 2026-01-14 22:19:29.236599
5 Eve True 2026-01-14 22:19:29.236599
6 Alice True 2026-01-15 16:27:08.117127
7 Bob True 2026-01-15 16:27:08.294889
8 Charlie True 2026-01-15 16:27:08.408266
9 Dave True 2026-01-15 16:27:08.538241
10 Eve True 2026-01-15 16:27:08.657943
11 Alice True 2026-01-16 20:44:47.109342
12 Bob True 2026-01-16 20:44:47.251003
13 Charlie True 2026-01-16 20:44:47.428983
14 Dave True 2026-01-16 20:44:47.551127
15 Eve True 2026-01-16 20:44:47.725328
16 Alice True 2026-01-17 23:07:04.702524
17 Bob True 2026-01-17 23:07:04.820589
18 Charlie True 2026-01-17 23:07:04.988969
19 Dave True 2026-01-17 23:07:05.112080
20 Eve True 2026-01-17 23:07:05.251115
21 Alice True 2026-01-19 16:39:33.754077
22 Bob True 2026-01-19 16:39:34.029185
23 Charlie True 2026-01-19 16:39:34.147111
24 Dave True 2026-01-19 16:39:34.255373
25 Eve True 2026-01-19 16:39:34.355807

In [14]:
users = session.query(User).filter(User.name == "Alice").all()
for user in users:
    print(user.id, user.name, user.is_active, user.created_at)

1 Alice True 2026-01-14 22:19:29.236599
6 Alice True 2026-01-15 16:27:08.117127
11 Alice True 2026-01-16 20:44:47.109342
16 Alice True 2026-01-17 23:07:04.702524
21 Alice True 2026-01-19 16:39:33.754077
26 Alice True 2026-01-25 22:13:31.654680
31 Alice True 2026-01-25 22:14:03.272966
36 Alice True 2026-01-25 22:14:47.012095
41 Alice True 2026-01-25 22:15:49.635300
46 Alice True 2026-01-26 16:33:20.852105


In [ ]:
from typing import List, Optional
from sqlalchemy.orm import Mapped, mapped_column, relationship
from sqlalchemy import ForeignKey


class Category(Base):
    __tablename__ = "categories"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(nullable=True)
    description: Mapped[Optional[str]] = mapped_column(nullable=True)
    products: Mapped[List["Product"]] = relationship(back_populates="category")

    def __repr__(self) -> str:
        return f"Category(id={self.id!r}, name={self.name!r}, description={self.description!r})"


class Product(Base):
    __tablename__ = "products"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(nullable=True)
    description: Mapped[str] = mapped_column(nullable=True)
    category_id: Mapped[int] = mapped_column(ForeignKey("categories.id"))

    category: Mapped[Category] = relationship(back_populates="products")

    def _repr__(self) -> str:
        return f"Product(id={self.id!r}, name={self.name!r}, description={self.description!r}, category_id={self.category_id!r})"

In [16]:
Base.metadata.create_all(engine)

In [17]:
categories = [
    Category(name="Electronics", description="Electronic items"),
    Category(name="Clothing", description="Clothing items"),
    Category(name="Books", description="Books and literature"),
]

for category in categories:
    session.add(category)
    session.commit()

products = [
    Product(name="Laptop", description="A personal computer", category_id=1),
    Product(name="Smartphone", description="A mobile phone", category_id=1),
    Product(name="T-Shirt", description="A cotton t-shirt", category_id=2),
    Product(name="Jeans", description="Denim jeans", category_id=2),
    Product(name="Novel", description="A fiction book", category_id=3),
    Product(name="Biography", description="A non-fiction book", category_id=3),
]

session.add_all(products)
session.commit()

In [18]:
categories = session.query(Category).all()
for category in categories:
    print(category)

products = session.query(Product).all()
for product in products:
    for key, value in product.__dict__.items():
        print(key, value)

Category(id=1, name='Electronics', description='Electronic items')
Category(id=2, name='Clothing', description='Clothing items')
Category(id=3, name='Books', description='Books and literature')
Category(id=4, name='Electronics', description='Electronic items')
Category(id=5, name='Clothing', description='Clothing items')
Category(id=6, name='Books', description='Books and literature')
Category(id=7, name='Electronics', description='Electronic items')
Category(id=8, name='Clothing', description='Clothing items')
Category(id=9, name='Books', description='Books and literature')
Category(id=10, name='Electronics', description='Electronic items')
Category(id=11, name='Clothing', description='Clothing items')
Category(id=12, name='Books', description='Books and literature')
Category(id=13, name='Electronics', description='Electronic items')
Category(id=14, name='Clothing', description='Clothing items')
Category(id=15, name='Books', description='Books and literature')
Category(id=16, name='El

In [19]:
from os import name


def print_products_details(products):
    for product in products:
        for key, value in product.__dict__.items():
            print(key, value)

def print_product_details(product):
    if product:
        for key, value in product.__dict__.items():
            print(key, value)

products = session.query(Product).filter(Product.name == "Laptop").all()
print_products_details(products)

products = session.query(Product).filter(Product.name.icontains("Laptop")).all()
print_products_details(products)

products = session.query(Product).filter(Product.category.has(name="Electronics")).all()
print_products_details(products)

products = session.query(Product).filter(Product.name.ilike("%phone%"), Product.category.has(name="Electronics")).all()
print_products_details(products)

product = session.query(Product).filter(Product.name.ilike("%top%"), Product.category.has(name="Electronics")).first()
print_product_details(product)

product = session.query(Product).filter_by(id=1, name="Laptop").first()
print_product_details(product)

_sa_instance_state <sqlalchemy.orm.state.InstanceState object at 0x000001C3EB644D60>
id 7
category_id 1
description A personal computer
name Laptop
_sa_instance_state <sqlalchemy.orm.state.InstanceState object at 0x000001C3EB645120>
id 18
category_id 1
description A personal computer
name Laptop
_sa_instance_state <sqlalchemy.orm.state.InstanceState object at 0x000001C3EB645360>
id 24
category_id 1
description A personal computer
name Laptop
_sa_instance_state <sqlalchemy.orm.state.InstanceState object at 0x000001C3EB6455A0>
id 30
category_id 1
description A personal computer
name Laptop
_sa_instance_state <sqlalchemy.orm.state.InstanceState object at 0x000001C3EB6457E0>
id 36
category_id 1
description A personal computer
name Laptop
_sa_instance_state <sqlalchemy.orm.state.InstanceState object at 0x000001C3EB645A20>
id 42
category_id 1
description A personal computer
name Laptop
_sa_instance_state <sqlalchemy.orm.state.InstanceState object at 0x000001C3EB645C60>
id 48
category_id 1
de

In [20]:
product = session.query(Product).filter(Product.id == 7).first()
product.name = "Laptop"
print(product)
session.commit()
product = session.query(Product).filter(Product.id == 7).first()
print_product_details(product)

_sa_instance_state <sqlalchemy.orm.state.InstanceState object at 0x000001C3EB646B60>
id 7
category_id 1
description A personal computer
name Laptop


In [24]:
product = session.query(Product).filter(Product.id == 4).first()
session.delete(product)
session.commit()

products = session.query(Product).all()
for product in products:
    for key, value in product.__dict__.items():
        if key == "name":
            print(value)

Novel
Biography
Laptop
Smartphone
T-Shirt
Jeans
Novel
Smartphone
T-Shirt
Jeans
Novel
Biography
Laptop
Smartphone
T-Shirt
Jeans
Novel
Biography
Laptop
Smartphone
T-Shirt
Jeans
Novel
Biography
Laptop
Smartphone
T-Shirt
Jeans
Novel
Biography
Laptop
Smartphone
T-Shirt
Jeans
Novel
Biography
Laptop
Smartphone
T-Shirt
Jeans
Novel
Biography
Laptop
Smartphone
T-Shirt
Jeans
Novel
Biography
Laptop
Smartphone
T-Shirt
Jeans
Novel
Biography


Ordering & Pagination

In [25]:
products = session.query(Product).order_by(Product.name.asc()).limit(2).offset(2)
print_products_details(products)

_sa_instance_state <sqlalchemy.orm.state.InstanceState object at 0x000001C3EC008340>
id 23
category_id 3
description A non-fiction book
name Biography
_sa_instance_state <sqlalchemy.orm.state.InstanceState object at 0x000001C3EC0083A0>
id 29
category_id 3
description A non-fiction book
name Biography


Aggregations

In [26]:
count = session.query(Product.category_id).distinct().count()
print("Distinct category count:", count)

Distinct category count: 3


In [27]:
from sqlalchemy import func



count = session.query(func.count(Product.category_id).distinct()).scalar()
print("Distinct category count:", count)

_sum = session.query(func.sum(Product.category_id)).scalar()
print("Sum of category IDs:", _sum)

avg = session.query(func.avg(Product.category_id)).scalar()
print("Average of category IDs:", avg)

_max = session.query(func.max(Product.category_id)).scalar()
print("Maximum of category IDs:", _max)

_min = session.query(func.min(Product.category_id)).scalar()
print("Minimum of category IDs:", _min)

count, avg, _sum, _max, _min = session.query(
    func.count(Product.category_id), 
    func.avg(Product.category_id),
    func.sum(Product.category_id),
    func.max(Product.category_id),
    func.min(Product.category_id),
).one()

print("Distinct category count:", count)
print("Average of category IDs:", avg)
print("Sum of category IDs:", _sum)
print("Maximum of category IDs:", _max)
print("Minimum of category IDs:", _min)

Distinct category count: 54
Sum of category IDs: 110
Average of category IDs: 2.037037037037037
Maximum of category IDs: 3
Minimum of category IDs: 1
Distinct category count: 54
Average of category IDs: 2.037037037037037
Sum of category IDs: 110
Maximum of category IDs: 3
Minimum of category IDs: 1


In [28]:
products = session.query(Product).all()
for product in products:
    print(product.category.name)

Books
Books
Electronics
Electronics
Clothing
Clothing
Books
Electronics
Clothing
Clothing
Books
Books
Electronics
Electronics
Clothing
Clothing
Books
Books
Electronics
Electronics
Clothing
Clothing
Books
Books
Electronics
Electronics
Clothing
Clothing
Books
Books
Electronics
Electronics
Clothing
Clothing
Books
Books
Electronics
Electronics
Clothing
Clothing
Books
Books
Electronics
Electronics
Clothing
Clothing
Books
Books
Electronics
Electronics
Clothing
Clothing
Books
Books


In [29]:
categories = session.query(Category).join(Category.products).distinct().all()

for category in categories:
    print(category.name, ": ", ", ".join([str(product.name) for product in category.products]))



Books :  Novel, Biography, Novel, Novel, Biography, Novel, Biography, Novel, Biography, Novel, Biography, Novel, Biography, Novel, Biography, Novel, Biography, Novel, Biography
Electronics :  Laptop, Smartphone, Smartphone, Laptop, Smartphone, Laptop, Smartphone, Laptop, Smartphone, Laptop, Smartphone, Laptop, Smartphone, Laptop, Smartphone, Laptop, Smartphone
Clothing :  T-Shirt, Jeans, T-Shirt, Jeans, T-Shirt, Jeans, T-Shirt, Jeans, T-Shirt, Jeans, T-Shirt, Jeans, T-Shirt, Jeans, T-Shirt, Jeans, T-Shirt, Jeans


In [40]:
categories = session.query(Category).join(Product)
for category in categories:
    print([product.__dict__ for product in category.products])
    for key, value in category.__dict__.items():
        print(key, value)

[{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001C3EBFEBEE0>, 'id': 5, 'category_id': 3, 'description': 'A fiction book', 'name': 'Novel', 'category': Category(id=3, name='Books', description='Books and literature')}, {'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001C3EBFEBF40>, 'id': 6, 'category_id': 3, 'description': 'A non-fiction book', 'name': 'Biography', 'category': Category(id=3, name='Books', description='Books and literature')}, {'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001C3EC06B280>, 'id': 11, 'category_id': 3, 'description': 'A fiction book', 'name': 'Novel', 'category': Category(id=3, name='Books', description='Books and literature')}, {'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001C3EC06AF80>, 'id': 16, 'category_id': 3, 'description': 'A fiction book', 'name': 'Novel', 'category': Category(id=3, name='Books', description='Books and literature')}, {'_sa